 # Comparing kNN Classification Capabilities with 3 Data Models

- Bag of Words
- TF-IDF
- TF-IDF with LSA

Dependency Installation: `pip install scikit-learn matplotlib numpy`

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

## Bag of Words Feature Extraction Function

This function takes a list of text documents and returns the the bag of words matrix using the `CountVectorizer` class from `scikit-learn`.

An explanation of how Bag of Words is calculated:

- Build a vocabulary from the collection of documents (limiting to a maximum number of features)

- For each document, represent it as a vector where each element is the count of a specific term from the vocabulary

- The resulting matrix has dimensions `m x n`, where `m` is the number of documents and `n` is the vocabulary size

In [21]:
def compute_bow(documents, max_features):
    """
    Computes the bag-of-words matrix for the given documents.
    
    Parameters:
    - documents: List of text documents.
    - max_features: Maximum number of features (vocabulary size).
    
    Returns:
    - bow_matrix: Sparse matrix of shape (n_samples, n_features).
    """
    vectorizer = CountVectorizer(max_features=max_features)
    bow_matrix = vectorizer.fit_transform(documents)
    return bow_matrix

## TF-IDF Feature Extraction

This function takes a list of text documents and returns the tfidf-weighted term-document matrix, using the `TfidfVectorizer` calss from `scikit-learn` to perform the calculations described below.

A reminder on how TF-IDF matrices are calculated:

- For each term `t` in document `d`, compute the term frequency: `tf(t, d)` = frequency of `t` in `d`

- Compute the inverse document frequency over all `N` documents: `idf(t) = log((N + 1) / (df(t) + 1)) + 1`, where `df(t)` is the number of documents containing `t`

- Calculate the TF-IDF weight for each term-document pair: `tfidf(t, d) = tf(t, d) * idf(t)`

In [23]:
def compute_tfidf(documents, max_features):
    """
    Computes the tfidf matrix for the given documents.
    
    Parameters:
    - documents: List of text documents.
    - max_features: Maximum number of features to use.
    
    Returns:
    - tfidf_matrix: Sparse matrix of shape (n_samples, n_features).
    """
    vectorizer = TfidfVectorizer(max_features=max_features)
    tfidf_matrix = vectorizer.fit_transform(documents)
    return tfidf_matrix

## Low-Rank Approximation via Latent Semantic Analysis (LSA)

This function uses the `TruncatedSVD` class from `scikit-learn` to reduce the dimensionality of the input tfidf matrix.

A reminder on how LSA is performed:

- Start with the TF-IDF matrix `A` of size `m x n` (with `m` documents and `n` terms)

- Apply Singular Value Decomposition (SVD): `A = U * Σ * Vᵀ`

- Create a low-rank approximation by retaining the top `k` singular values and corresponding vectors: `A ≈ Uₖ * Σₖ * Vₖᵀ`, where `Σₖ` is a `k x k` diagonal matrix

In [24]:
def apply_lsa(tfidf_matrix, n_components):
    """
    Applies LSA (using TruncatedSVD) to the tfidf matrix.
    
    Parameters:
    - tfidf_matrix: Sparse matrix from tfidf vectorization.
    - n_components: Number of components to keep.
    
    Returns:
    - lsa_matrix: Dense matrix with reduced dimensions.
    """
    svd = TruncatedSVD(n_components=n_components)
    lsa_matrix = svd.fit_transform(tfidf_matrix)
    return lsa_matrix

## kNN Classification

This function trains a kNN classifier on the input features using the `KNeighborsClassifier` class from `scikit-learn` and calculates its acurracy.

A reminder on how KNN training is performed:

- Represent each document by its feature vector (from either the TF-IDF matrix or the LSA-reduced matrix)

- Compute the Euclidean distance between vectors: `d(x, xᵢ) = || x - xᵢ ||₂`

- For a given document, identify the `k` nearest neighbors based on these distances

- Assign the document to the class most common among these `k` neighbors

In [33]:
def evaluate_knn(features, labels, n_neighbors=5, test_size=0.2):
    """
    Splits the dataset, trains a kNN classifier, and evaluates performance.
    
    Parameters:
    - features: Feature matrix (can be tfidf or LSA-transformed).
    - labels: Ground truth labels.
    - n_neighbors: Number of neighbors for kNN.
    - test_size: Fraction of data to use for testing.
    
    Returns:
    - accuracy: Accuracy score on the test set.
    """
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=test_size)

    neigh = KNeighborsClassifier(n_neighbors=n_neighbors)
    neigh.fit(X_train, y_train)

    y_pred = neigh.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy

## Load Test Data

- 20newsgroups dataset (see here for more [info](https://www.kaggle.com/datasets/crawford/20-newsgroups))

  - Text which is categorized into 20 different categories

  - `newsgroups.data` contains the text data

  - `newsgroups.target` contains the corresponding categories

In [11]:
# Load 20 Newsgroups dataset (using all available data for a robust comparison)
categories = ['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
newsgroups = fetch_20newsgroups(subset='all', categories=categories, 
                                remove=('headers', 'footers', 'quotes'))
documents = newsgroups.data
labels = newsgroups.target
num_features = 10000 # Not too many for speed
num_components = 100

# Apply All 3 Data Models with kNN Classifier

In [29]:
# --- Pipeline 1: Bag-of-Words ---
bow_matrix = compute_bow(documents, num_features)
acc_bow = evaluate_knn(bow_matrix, labels)
print(f"\nEvaluating kNN classifier on Bag-of-Words features: {acc_bow:.2f} acurracy")


# --- Pipeline 2: TF-IDF ---
tfidf_matrix = compute_tfidf(documents, num_features)
acc_tfidf = evaluate_knn(tfidf_matrix, labels)
print(f"\nEvaluating kNN classifier on TF-IDF features: {acc_tfidf:.2f} acurracy")


# --- Pipeline 3: TF-IDF + LSA ---
lsa_matrix = apply_lsa(tfidf_matrix, num_components)
acc_lsa = evaluate_knn(lsa_matrix, labels)
print(f"\nEvaluating kNN classifier on LSA-transformed features: {acc_lsa:.2f} acurracy")


Evaluating kNN classifier on Bag-of-Words features: 0.45 acurracy

Evaluating kNN classifier on TF-IDF features: 0.32 acurracy

Evaluating kNN classifier on LSA-transformed features: 0.68 acurracy


## Comprehension Questions

1.	Why does applying LSA to the TF-IDF matrix significantly improve the kNN classifier’s performance compared to using raw TF-IDF or Bag-of-Words features?

It helps by capturing the underlying relationships between the words and documents. SVD is used to reduce dimensionality and gropup similar terms together, which can help knn perform better.

2.	Given that the TF-IDF approach resulted in lower accuracy than the Bag-of-Words model in this example, what factors could be contributing to this outcome, and what insights does it offer about the weighting of terms in text classification?

TF-IDF reduces the weight for the frequent words, giving more importance to rare ones. This might not be good for kNN since reducing the influence of frequent words can cause incorrect nearest neighbor matches. It tells us that term weighting affects text classification performance, and depending how what is used, different weighing approaches might be more effective. In the case of TF-IDF and kNN, it wasn't as helpful in improving the accuracy in the final model. 